In [1]:
%load_ext lab_black

In [2]:
import torch
from torch import nn

In [4]:
from functools import partial


def add_func(a, b, c):
    print(f"{a}+{b}+{c}")
    return a + b + c


add_list = list(map(partial(add_func, b=5, c=3), [0, 1, 2, 3, 4, 5]))

print(add_list)

0+5+3
1+5+3
2+5+3
3+5+3
4+5+3
5+5+3
[8, 9, 10, 11, 12, 13]


In [22]:
x = torch.rand((1, 3, 64, 64))
upsample = lambda x: torch.nn.functional.interpolate(x, scale_factor=2)
up_x = upsample(x)

In [29]:
class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(ResBlock, self).__init__()

        self.c0 = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.c1 = nn.Conv2d(out_ch, out_ch, 3, 1, 1)
        self.c_sc = nn.Conv2d(in_ch, out_ch, 1, 1, 0)

        self.bn0 = nn.BatchNorm2d(in_ch)
        self.bn1 = nn.BatchNorm2d(out_ch)

        self.activation = nn.ReLU(inplace=False)

        self.upsample = lambda x: torch.nn.functional.interpolate(x, scale_factor=2)
        self.learnable_sc = in_ch != out_ch

    def residual(self, x):
        h = self.bn0(x)
        h = self.activation(h)
        h = self.c0(h)
        h = self.upsample(h)
        h = self.bn1(h)
        h = self.activation(h)
        h = self.c1(h)
        return h

    def shortcut(self, x):
        if self.learnable_sc:
            x = self.upsample(x)
            x = self.c_sc(x)
            return x
        else:
            return x

    def foward(self, x):
        return self.residual(x) + self.shortcut(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self,ngpu):
        super(Generator,self).__init__()
        self.ngpu = ngpu
        
        self.l0 = nn.Linear(128,4*4*16)
        self.res1 = ResBlock(16,16)
        self.res1 = ResBlock(16,8)
        self.res1 = ResBlock(8,8)
        self.res1 = ResBlock(8,4)
        self.res1 = ResBlock(4,2)
    def forward(self,input):
        return 

In [ ]:
class Discriminator(nn.Module):
    def __init__(self,ngpu):
        super(Discriminator,self).__init__()
        self.ngpu = ngpu
        
    def forward(self,input):
        

In [3]:
def weight_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)